In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import random
from PIL import Image


#特征的维度最大化
#

/Users/dylan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
r'''
1、先导入视频
2、将视频分成帧序列
3、随机选取帧，且帧的位置在视频序列中的中间帧之前
4、获取帧的感知哈希数
5、从该帧开始依次获取每一帧的感知哈希数，并与该帧作对比，若第n帧的感知哈希数近似值大于阈值，则判定为相同帧
6、将开始帧与第n帧之间的帧序列导出为关键帧
7、将关键帧序列传入神经网络进行训练，并进行分类

需要调参的因素有：
1、图片缩小之后的大小，如20 * 20 = 400个像素点
2、汉明距离：hanming_distance的值，这个值越大，说明两帧像素之间的差别值越多，则满足这个汉明距离的帧图像也就越多
3、每个关键帧视频中的帧的个数，keyframe_num的值，这个值越大，说明一段关键帧视频的中的帧的数目越多。
   当这个数值较小时，无法获取一个完整的手势动作，当数值较大时，会造成冗余状态，可能会重复。
4、随机帧的取值范围，即frame_random_count = random.randint(int(frame_count/4),int(frame_cont/2))

'''


'\n1、先导入视频\n2、将视频分成帧序列\n3、随机选取帧，且帧的位置在视频序列中的中间帧之前\n4、获取帧的感知哈希数\n5、从该帧开始依次获取每一帧的感知哈希数，并与该帧作对比，若第n帧的感知哈希数近似值大于阈值，则判定为相同帧\n6、将开始帧与第n帧之间的帧序列导出为关键帧\n7、将关键帧序列传入神经网络进行训练，并进行分类\n\n需要调参的因素有：\n1、图片缩小之后的大小，如20 * 20 = 400个像素点\n2、汉明距离：hanming_distance的值，这个值越大，说明两帧像素之间的差别值越多，则满足这个汉明距离的帧图像也就越多\n3、每个关键帧视频中的帧的个数，keyframe_num的值，这个值越大，说明一段关键帧视频的中的帧的数目越多。\n   当这个数值较小时，无法获取一个完整的手势动作，当数值较大时，会造成冗余状态，可能会重复。\n4、随机帧的取值范围，即frame_random_count = random.randint(int(frame_count/4),int(frame_cont/2))\n\n'

In [3]:
#3、获取随机帧的感知哈希数值(差值感知哈希算法)
#将图像缩小为8*8
def dHash(frame):
    #''' calutalate the Hash value of frame image'''
    copy_frame = np.zeros(frame.shape,np.uint8)
    copy_frame = frame.copy()
    re_frame = cv.resize(copy_frame,(20,20))

#转换灰度图
    gray_frame = cv.cvtColor(re_frame,cv.COLOR_BGR2GRAY)
    hash_str = ''
#每行前一个像素大于后一个像素为1，相反为0，生成哈希
    for i in range(20):
        for j in range(19):
            if gray_frame[i,j] > gray_frame[i,j+1]:
                hash_str = hash_str + '1'
            else:
                hash_str = hash_str + '0'
    hash_str = hash_str + '0'
    return hash_str

In [4]:
#4、计算汉明距离
def cmpHash(dHash1,dHash2):
    hanming_distance = 0
    for i in range(len(dHash1)):
        if dHash1[i] != dHash2[i]:
            hanming_distance = hanming_distance + 1
    return hanming_distance


In [10]:
cap = cv.VideoCapture('/Users/dylan/anaconda3/Data/video/IMG_0556.MOV')
#导入视频
if cap.isOpened():
    print('success')
    exit(1)
else:
    print('can\'t find the video !')

#1、读取每帧的视频图像
frame_width = int(cap.get(3))
frame_length = int(cap.get(4))
#fps = cv.videoCapture.get(cv.CV_CAP_PROP_FPS)
keyframe_out = cv.VideoWriter('keyframe_out1.mov',
                              cv.VideoWriter_fourcc('M', 'J', 'P', 'G'), 
                              20, 
                              (frame_width, frame_length))

print('该视频中的每帧的长度为：' + str(frame_length))
print('该视频中的每帧的宽度为：' + str(frame_width))

success
该视频中的每帧的长度为：1080
该视频中的每帧的宽度为：1920


In [11]:
#2、获取到随机帧
#获取视频的总帧数目以及随机帧
frame_count = 0
ret,frame = cap.read()
#当读取到最后一帧时，ret值为零
while ret:
    frame_count = frame_count + 1
    ret,frame = cap.read()
    
#从总视频帧的1/4到1/2之间取随机帧，避免获取到无关帧以及超出关键帧的长度。
frame_random_count = random.randint(int(frame_count/4),int(frame_count/2))
#将视频的初始帧定位到随机帧的位置
cap.set(cv.CAP_PROP_POS_FRAMES,frame_random_count)
#frame_random_width = int(cap.get(3))
#frame_random_length = int(cap.get(4))

#以上完成



#问题在于从随机帧开始，只能获取到一帧的图像便结束了




#获取到随机帧的第一帧
ret,frame = cap.read()
frame_width = int(cap.get(3))
frame_length = int(cap.get(4))
flip_frame = cv.flip(frame, 0)


first_frame = np.zeros(flip_frame.shape,np.uint8)
first_frame = flip_frame.copy()
    
keyframe_out.write(first_frame)#将随机帧写入输出视频的序列
print('这是第一帧')
print(dHash(first_frame))
print('这是分割线')
print('该视频中的总帧数为：' + str(frame_count))
print('随机获得的帧为第' + str(frame_random_count) + '帧')



#print(dHash(next_frame))

print('该视频中的每帧的长度为：' + str(frame_length))
print('该视频中的每帧的宽度为：' + str(frame_width))


这是第一帧
110000111111111111111000011111111111110100001111111111111110000111111111111111000011111111111111100000111111111111110000111111111111111000111111111111111100110001111111111110011110011111111110011011100111111111111110011011111001111011001001111111101101110000111111110011001010111111111100100100111111111100101100100111111110101100010000111101000110000011001111100000000000000011110
这是分割线
该视频中的总帧数为：189
随机获得的帧为第87帧
该视频中的每帧的长度为：1080
该视频中的每帧的宽度为：1920


In [12]:
i = 1
keyframe_num = 0
#获取关键帧之后的帧图像
#将获取到的帧图像依次与随机帧图像进行对比
#若不是相同帧则将不相同的图像帧存储到序列中，否则结束帧的读取（使用break语句来结束）
while ret:
    ret,frame = cap.read()#这里是不是又重新读取了一遍第一帧呢?实验得出，确实读取了相同的帧
    if i == 1 :                #只运行一次，将视频读取到第二帧
        i = i - 1
        ret,frame = cap.read()
        next_flip_frame = cv.flip(frame, 0)
    if cmpHash(dHash(first_frame),dHash(next_flip_frame)) < 10:
        #keyframe_out.write(next_frame)
        print('这是cmp的break')
        break
    keyframe_out.write(next_flip_frame)
    if keyframe_num > 25:
        print('这是keyframe_num的break')
        break
    keyframe_num = keyframe_num + 1
    print(dHash(next_flip_frame))
print('结束')

110001111111111111111000011111111111111100001111111111111110000111111111111111000011111111111111100000111111111111110000011111111111111000011111111111111100011001111111111110011100011111111110011111000111111111011111101011111001111011000000111111101111010101011111111011001010111111111100100100111111110100101100010111111110001110000001111111000110000001000111100000000000010111110
110001111111111111111000011111111111111100001111111111111110000111111111111111000011111111111111100000111111111111110000011111111111111000011111111111111100011001111111111110011100011111111110011111000111111111011111101011111001111011000000111111101111010101011111111011001010111111111100100100111111110100101100010111111110001110000001111111000110000001000111100000000000010111110
11000111111111111111100001111111111111110000111111111111111000011111111111111100001111111111111110000011111111111111000001111111111111100001111111111111110001100111111111111001110001111111111001111100011111111101111110101111100111101100